#### Duckdb for data science

Dataset:

https://web.stanford.edu/class/cs102/datasets/Titanic.csv

In [23]:
# import libraries

import pandas as pd
import duckdb
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import time

In [4]:
# dataset

,last,first,gender,age,class,fare,embarked,survived
0,Braund,Mr. Owen Harris,M,22.0,3,7.2500,Southampton,no
1,Cumings,Mrs. John Bradley (Florence Briggs Thayer),F,38.0,1,71.2833,Cherbourg,yes
2,Heikkinen,Miss Laina,F,26.0,3,7.9250,Southampton,yes
3,Futrelle,Mrs. Jacques Heath (Lily May Peel),F,35.0,1,53.1000,Southampton,yes
4,Allen,Mr. William Henry,M,35.0,3,8.0500,Southampton,no
...,...,...,...,...,...,...,...,...
886,Montvila,Rev. Juozas,M,27.0,2,13.0000,Southampton,no
887,Graham,Miss Margaret Edith,F,19.0,1,30.0000,Southampton,yes
888,Johnston,"Miss Catherine Helen ""Carrie""",F,NaN,3,23.4500,Southampton,no
889,Behr,Mr. Karl Howell,M,26.0,1,30.0000,Cherbourg,yes


Pandas

In [28]:
# Load dataset using Pandas

start_time_pandas = time.time()
df_pandas = pd.read_csv('titanic.csv')

# Preprocess data (simple example)
df_pandas = df_pandas[['class', 'gender', 'age', 'fare', 'survived']]
df_pandas['gender'] = df_pandas['gender'].map({'M': 0, 'F': 1})
df_pandas['age'].fillna(df_pandas['age'].median(), inplace=True)

# Train/test split
X = df_pandas.drop('survived', axis=1)
y = df_pandas['survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit Random Forest model using Pandas data
model_pandas = RandomForestClassifier(n_estimators=100, random_state=42)
model_pandas.fit(X_train, y_train)
predictions_pandas = model_pandas.predict(X_test)
accuracy_pandas = accuracy_score(y_test, predictions_pandas)

time_pandas = time.time() - start_time_pandas
print(f'Time using Pandas: {time_pandas} seconds')

Time using Pandas: 0.11351895332336426 seconds


Duckdb

In [27]:

# create Duckdb connection

start_time_duckdb = time.time()
con = duckdb.connect()

# SQL query to preprocess data directly in DuckDB
query = """
SELECT
    class,
    CASE WHEN gender = 'M' THEN 0 ELSE 1 END AS gender,
    COALESCE(age, FAVG(age)) AS age,
    fare,
    survived
FROM
    read_csv_auto('titanic.csv')
GROUP BY class, gender, age, fare, survived
"""

# Create a DuckDB table from the query
df_duckdb = con.execute(query).fetchdf()

# Train/test split for DuckDB DataFrame
X_duckdb = df_duckdb.drop('survived', axis=1)
y_duckdb = df_duckdb['survived']
X_train_duckdb, X_test_duckdb, y_train_duckdb, y_test_duckdb = train_test_split(X_duckdb, y_duckdb, test_size=0.2, random_state=42)

# Fit Random Forest model using DuckDB data
model_duckdb = RandomForestClassifier(n_estimators=100, random_state=42)
model_duckdb.fit(X_train_duckdb, y_train_duckdb)
predictions_duckdb = model_duckdb.predict(X_test_duckdb)
accuracy_duckdb = accuracy_score(y_test_duckdb, predictions_duckdb)
time_duckdb = time.time() - start_time_duckdb

print(f'Time using Duckdb: {time_duckdb} seconds')

# Clean up
con.close()

Time using Duckdb: 0.12395358085632324 seconds
